# 使用 LangChain 示例
sk-c64994ca9f9741888dc7bccdef2aa0a1

### 1. 获取 API key 并创建调用 API 的函数

首先使用`pip install dashscope`安装阿里云百炼的 Python SDK

然后导入相关依赖包，使用`getpass`输入api_key

In [2]:
from getpass import getpass
from dashscope import Generation
import os
# 安全获取API Key（输入时不显示明文）
api_key = getpass("请输入阿里云百炼API Key: ")
os.environ["DASHSCOPE_API_KEY"] = api_key
Generation.api_key = api_key


In [3]:
def call_qwen(prompt="请介绍一下通义千问模型", model="qwen-turbo"):
    """
    调用通义千问模型获取回答

    Args:
        prompt: 用户输入的问题
        model: 使用的模型名称，默认为"qwen-turbo"

    Returns:
        模型生成的回答内容
    """
    try:
        # 调用模型生成文本（单轮对话用prompt参数）
        response = Generation.call(
            model="qwen-turbo",  # 指定模型
            prompt=prompt,  # 单轮对话使用prompt
            api_key=api_key  # 使用环境变量中的API Key
        )

        # 解析响应（output是字典，用["text"]获取内容）
        if response.status_code == 200:
            return response.output["text"]  # 修正解析方式
        else:
            return f"请求失败，状态码: {response.status_code}，原因: {response.message}"
    except Exception as e:
        return f"调用出错: {str(e)}"

# 测试调用
print(call_qwen("通义千问模型的特点是什么？"))

通义千问（Qwen）是通义实验室研发的超大规模语言模型，具有以下特点：

1. **强大的语言理解与生成能力**：通义千问能够理解和生成多种语言的文本，支持多轮对话、文本续写、问答等多种任务。

2. **广泛的知识覆盖**：模型在训练过程中学习了大量文本数据，因此在多个领域都有较深的知识积累，可以回答各种问题。

3. **多语言支持**：除了中文，通义千问还支持英文、法语、西班牙语、葡萄牙语、俄语、日语、韩语等十余种语言。

4. **良好的对话表现**：通义千问在对话中能够保持上下文连贯，理解用户意图，并提供自然流畅的回复。

5. **可定制化**：用户可以根据具体需求对模型进行微调，以适应特定的应用场景。

6. **高性能**：通义千问在多个基准测试中表现出色，如GLUE、SuperGLUE、MMLU等，展示了其强大的性能。

7. **安全与合规**：通义千问在设计和训练过程中注重安全性和合规性，能够识别并避免生成违法不良信息。

8. **开源与商业应用**：通义千问提供了多种版本，包括开源版本和商业版本，满足不同用户的需求。

总之，通义千问是一个功能强大、应用广泛的大型语言模型，适用于多种场景和任务。


### 后端接口设计

- 导入本地文件
  - .txt
  - .md
  - .pdf
- 根据主题检索知识库文段
- 文段导入大模型生成QA问答
  - 单个文段导入大模型
  - 循环生成
- 处理生成的QA问答实现数据格式化
- 保存生成的题库至本地


In [4]:
###导入本地文件函数
def import_local_file(file_path):
    """
    导入本地文件内容

    Args:
        file_path: 本地文件路径

    Returns:
        成功导入文件内容或错误信息
    """



In [5]:
###

### 2. 使用 LangChain 调用通义千问模型

In [6]:
from langchain_community.llms import Tongyi
# from langchain_community.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [7]:
# 初始化模型
llm = Tongyi(
    model="qwen-turbo",
    temperature=0.7,
    max_tokens=1000,
    api_key=api_key
)

question = "什么是人工智能？"

# 创建提示模板
template = """
你是一个有用的助手。请回答以下问题：

问题：{question}

回答：
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

# 创建链
chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\12066\AppData\Local\Temp\ipykernel_18952\3351139764.py:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [8]:
# 调用模型
question = "什么是人工智能？"
response = chain.invoke({"question": question})
print(response)

{'question': '什么是人工智能？', 'text': '人工智能（Artificial Intelligence，简称AI）是指由人创造的能够模拟人类智能行为的系统或机器。它可以通过学习、推理、感知、语言理解、问题解决等能力，完成一些通常需要人类智慧才能完成的任务。\n\n人工智能可以分为两大类：\n\n1. **弱人工智能（Narrow AI）**：专注于执行特定任务，如语音识别、图像识别、推荐系统等。目前大多数实际应用都属于这一类，例如语音助手（如Siri）、人脸识别系统等。\n\n2. **强人工智能（General AI）**：具备与人类相当的全面认知能力，能够理解、学习和应用知识到各种不同领域。目前尚未实现，仍处于研究阶段。\n\n人工智能的核心技术包括机器学习、深度学习、自然语言处理、计算机视觉、机器人技术等。它广泛应用于医疗、金融、交通、教育、娱乐等多个领域，正在深刻改变我们的生活和工作方式。'}


### 3. 使用 faiss 构建RAG检索

使用`pip install pypdf faiss-cpu`安装依赖包

#### 3.1 初步创建向量数据库，根据所选文件进行切分和向量化

In [18]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings import DashScopeEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

os.environ["DASHSCOPE_API_KEY"] = api_key

def create_vector_store(file_path):

    if file_path.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith('.txt'):
        loader = TextLoader(file_path, encoding='utf-8')
    else:
        raise ValueError(f"不支持的文件类型: {file_path}")

    pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    docs = text_splitter.split_documents(pages)

    # 创建通义千问embeddings
    embeddings = DashScopeEmbeddings(
        model="text-embedding-v1"
    )

    vector_store = FAISS.from_documents(docs, embeddings)

    vector_store.save_local("faiss_index")

    print(f"成功创建RAG向量数据库，共处理 {len(docs)} 个文档片段")

#### 3.2 加载已有的向量数据库并添加新文档

In [ ]:


def load_new_documents(file_path):
    embeddings = DashScopeEmbeddings(model="text-embedding-v1")
    vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

    if file_path.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith('.txt'):
        loader = TextLoader(file_path, encoding='utf-8')
    else:
        raise ValueError(f"不支持的文件类型: {file_path}")
    
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )
    docs = text_splitter.split_documents(documents)
    return docs


In [ ]:
load_new_documents

#### 3.3 根据关键词检索数据库

In [ ]:
def search_similar_documents(query, k=4):
    """
    根据查询词搜索相似文档
    query: 查询关键词或句子
    k: 返回最相似的k个结果
    """
    embeddings = DashScopeEmbeddings(model="text-embedding-v1")
    vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

    similar_docs = vector_store.similarity_search(query, k=k)
    
    print(f"查询: {query}")
    print(f"找到 {len(similar_docs)} 个相关文档:\n")
    
    for i, doc in enumerate(similar_docs):
        print(f"--- 文档 {i+1} ---")
        print(f"内容: {doc.page_content[:200]}...")
        print(f"来源: {doc.metadata.get('source', 'Unknown')}")
        print(f"页码: {doc.metadata.get('page', 'N/A')}")
        print("-" * 50)
    
    return similar_docs

# 使用示例
query = "延长学制"
results = search_similar_documents(query, k=3)

查询: 延长学制
找到 3 个相关文档:

--- 文档 1 ---
内容: — 13 —
阶段）；临床医学八年制本博连读生的博士研究生阶段基本修业
年限为4 年。
基本修业年限由各学科专业在本办法规定的年限内予以明
确。“强基计划”等本研贯通项目录取转段的研究生基本修业年
限要求依据相关项目培养规定确定。
第二十八条 研究生在基本修业年限内不能完成学业的，可
申请延长修业年限（以下简称延期）。全日制博士研究生最长修
业年限为在原基本修业年限基础上延长 2 年；全日制硕士研...
来源: ./data/sample.pdf
页码: 12
--------------------------------------------------
--- 文档 2 ---
内容: — 22 —
包括本数；所称的“不满”“超过”，不包括本数。
第五十六条 本办法由研究生院负责解释。
第五十七条 本办法自发布之日起施行，《浙江大学研究生
学籍管理实施办法》（浙大发研〔2017〕115 号）同时废止。本
办法实施之前已入学的研究生，可以继续适用原办法。
抄送：纪委，各院级党委，党委各部门，各党工委，工会、团委。
浙江大学校长办公室 主动公开 2024 年8 月16 日印发...
来源: ./data/sample.pdf
页码: 21
--------------------------------------------------
--- 文档 3 ---
内容: — 2 —
浙江大学研究生学籍管理实施办法
第一章 总 则
第一条 为全面贯彻国家教育方针，规范研究生学籍管理，
维护研究生正常教育教学秩序，保障研究生身心健康与合法权
益，促进研究生全面发展，依据教育部《普通高等学校学生管理
规定》（中华人民共和国教育部令第 41 号），制定本办法。
第二条 本办法适用于学校对接受普通高等学历教育的研
究生的管理。
第二章 新生入学
第三条 研究生新生应当在学校...
来源: ./data/sample.pdf
页码: 1
--------------------------------------------------


### 4. 将生成的题库保存到本地

In [10]:
import json
import uuid
from datetime import datetime

# 1. 定义保存路径
SAVE_DIR = "./question_banks"
import os
os.makedirs(SAVE_DIR, exist_ok=True)  # 确保文件夹存在

# 2. 生成单题数据
def create_question(stem, type_, answer, **kwargs):
    return {
        "question_id": str(uuid.uuid4()),  # 自动生成唯一ID
        "stem": stem,
        "type": type_,
        "answer": answer,
        "options": kwargs.get("options", []),
        "explanation": kwargs.get("explanation", ""),
        "difficulty": kwargs.get("difficulty", "medium"),
        "keywords": kwargs.get("keywords", []),
        "source": kwargs.get("source", ""),
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "updated_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

# 3. 保存题库到JSON
def save_bank_to_json(bank_name, questions, description=""):
    bank = {
        "bank_id": f"{bank_name.replace(' ', '_').lower()}_bank",
        "name": bank_name,
        "description": description,
        "questions": questions,
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "updated_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    file_path = os.path.join(SAVE_DIR, f"{bank_name}.json")
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(bank, f, ensure_ascii=False, indent=2)  # 缩进2空格，可读性强
    return file_path

# 4. 从JSON加载题库（用于刷题调用）
def load_bank_from_json(bank_name):
    file_path = os.path.join(SAVE_DIR, f"{bank_name}.json")
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

# 示例：生成并保存一个题库
if __name__ == "__main__":
    # 创建2道题
    q1 = create_question(
        stem="线性回归的目标是最小化什么损失？",
        type_="single_choice",
        answer="A",
        options=["平方损失", "交叉熵损失", "绝对值损失"],
        difficulty="easy",
        keywords=["线性回归", "损失函数"],
        source="机器学习入门.pdf"
    )
    # 保存题库
    save_bank_to_json("机器学习基础题库", [q1])
    # 加载题库（刷题时用）
    bank = load_bank_from_json("机器学习基础题库")
    print("加载的题目：", bank["questions"][0]["stem"])

加载的题目： 线性回归的目标是最小化什么损失？


### 5. 使用 gradio 实现前端界面

In [11]:
import gradio as gr



# 创建Gradio聊天界面
demo = gr.ChatInterface(
    fn=call_qwen,  # 传入函数对象，而非调用结果
    title="通义千问对话助手",
    description="与通义千问模型进行对话，支持多轮交流"
    
)

# 启动界面
if __name__ == "__main__":
    demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7860

Could not create share link. Missing file: d:\anaconda\envs\chatgpt\lib\site-packages\gradio\frpc_windows_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.2
3. Move the file to this location: d:\anaconda\envs\chatgpt\lib\site-packages\gradio
